## import package

In [1]:
import torch
import torch.nn as nn
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
import math
import numpy as np
import  time
import torch, pandas as pd
import nltk
import re
nltk.download('punkt')

from transformers import set_seed
set_seed(123)

[nltk_data] Downloading package punkt to /home/joeyliang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2022-12-01 00:11:54.923526: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-01 00:11:55.116542: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-01 00:11:55.736012: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib64:
2022-12-01 00:11:55.736089: W tensorfl

In [2]:
# Training data file
df=pd.read_csv("data/Batch_answers - train_data (no-blank) (1).csv", encoding = "ISO-8859-1")

In [3]:
df = df.drop(columns=['Unnamed: 6', 'total no.: 7987'])

In [4]:
df = df[df['s']!='AGREE']

In [5]:
df.head()

,id,q,r,s,q',r'
11,11,"""First , there is no `` us `` on your part reg...","""Ah , I see . Your reasons are secret reasons ...",DISAGREE,"""no one owes you anything""","""find that appalling"""
12,11,"""First , there is no `` us `` on your part reg...","""Ah , I see . Your reasons are secret reasons ...",DISAGREE,"""No one owes you anything here . As for me , I...","""Your reasons are secret reasons . On a debate..."
13,11,"""First , there is no `` us `` on your part reg...","""Ah , I see . Your reasons are secret reasons ...",DISAGREE,"""You should be giving justification for wantin...","""Your reasons are secret reasons . On a debate..."
14,11,"""First , there is no `` us `` on your part reg...","""Ah , I see . Your reasons are secret reasons ...",DISAGREE,"""do n't owe you any justification . Take your ...","""Your reasons are secret reasons . On a debate..."
15,12,"""alright how did evolution start ?""","""See above""",DISAGREE,"""how evolution start""","""above"""


## Data process

In [6]:
df[['q','r',"q'","r'"]] = df[['q','r',"q'","r'"]].apply(lambda x: x.str.strip('\"'))

In [7]:
df['r'] = df['s'] + ':' + df['r']

In [8]:
df.head()

,id,q,r,s,q',r'
11,11,"First , there is no `` us `` on your part rega...","DISAGREE:Ah , I see . Your reasons are secret ...",DISAGREE,no one owes you anything,find that appalling
12,11,"First , there is no `` us `` on your part rega...","DISAGREE:Ah , I see . Your reasons are secret ...",DISAGREE,"No one owes you anything here . As for me , I ...",Your reasons are secret reasons . On a debate ...
13,11,"First , there is no `` us `` on your part rega...","DISAGREE:Ah , I see . Your reasons are secret ...",DISAGREE,You should be giving justification for wanting...,Your reasons are secret reasons . On a debate ...
14,11,"First , there is no `` us `` on your part rega...","DISAGREE:Ah , I see . Your reasons are secret ...",DISAGREE,do n't owe you any justification . Take your s...,Your reasons are secret reasons . On a debate ...
15,12,alright how did evolution start ?,DISAGREE:See above,DISAGREE,how evolution start,above


In [9]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(df, test_size=0.2)
valid, test = train_test_split(valid, test_size=0.5)

## Tokenizer

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("nreimers/MiniLM-L6-H384-uncased")

In [11]:
train_data_q = train['q'].tolist()
valid_data_q = valid['q'].tolist()
test_data_q = test['q'].tolist()

train_data_r = train['r'].tolist()
valid_data_r = valid['r'].tolist()
test_data_r = test['r'].tolist()

train_encodings = tokenizer(train_data_q, train_data_r, truncation=True, padding=True)
val_encodings = tokenizer(valid_data_q, valid_data_r, truncation=True, padding=True)
test_encodings = tokenizer(test_data_q, test_data_r, truncation=True, padding=True)

In [12]:
train_data_q1 = train['q\''].tolist()
valid_data_q1 = valid['q\''].tolist()
test_data_q1 = test['q\''].tolist()

train_data_r1 = train['r\''].tolist()
valid_data_r1 = valid['r\''].tolist()
test_data_r1  = test['r\''].tolist()

label_train = tokenizer(train_data_q1, train_data_r1, truncation=True, padding=True)
label_valid = tokenizer(valid_data_q1, valid_data_r1, truncation=True, padding=True)
label_test  = tokenizer(test_data_q1, test_data_r1, truncation=True, padding=True)

# build label

In [13]:
tokenizer.decode(train_encodings['input_ids'][1])

"[CLS] i have characterized certain ideas, notions, and opinions as stupid and simple - minded, but not you personally. [SEP] disagree : oh, ww, come on now. if you say something to me about how vitality works, and i say'that's a stupid / simple - minded idea ', are you seriously telling me you wouldn't take that as a personal attack? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA

In [14]:
#label
label_to_id = {'pad':0, 'q_agreement':1, 'r_agreement':2}

In [15]:
def add_token_positions(df, encodings):
    agreements=[]
    for i in df['s']:
        if i =="AGREE":
            agreement=True
        else:
            agreement=False
        agreements.append(agreement)
    encodings.update({'agreement':agreements})
add_token_positions(train, train_encodings)
add_token_positions(valid, val_encodings)
add_token_positions(test, test_encodings)

In [16]:
from tensorflow.keras.utils import to_categorical
from tqdm.contrib import tzip
def add_token_positions(encodings, label_encodings):
    label=[]
        
    for encoding1, encoding2  in tzip(encodings['input_ids'],label_encodings['input_ids']):
        
        sentence1 = encoding1.copy()
        sentence2 = encoding2.copy()
        sentence2 = [i for i in sentence2 if i != 0]   # PAD
        sentence2 = [i for i in sentence2 if i != 101] # CLS
        sentence2 = [i for i in sentence2 if i != 102] # SEP
        
        # build ner label
        origin_label = []
        length = len(sentence1)
        r=False
        while(len(origin_label)!=length):
            if len(sentence2)==0:                             # q_sentence2 is empty, already found in q_sentence1
                for i in range(len(sentence1)):
                    origin_label.append(label_to_id['pad'])
                break
                
            if sentence1[0]==0 or sentence1[0]==101 or sentence1[0] ==102: # CLS PAD SEP
                if sentence1[0] ==102:
                    r=True
                origin_label.append(label_to_id['pad'])
                del sentence1[0]
                
            else:
                if sentence1[0]!=sentence2[0]:      # not same
                    origin_label.append(label_to_id['pad'])
                    del sentence1[0]
                    
                else:                               # same
                    if r == True:
                        origin_label.append(label_to_id['r_agreement'])
                    else:
                        origin_label.append(label_to_id['q_agreement'])
    
                    del sentence1[0]
                    del sentence2[0] 
                    
        origin_label = [to_categorical(i, num_classes=len(label_to_id)) for i in origin_label]
        label.append(origin_label)
        
    encodings.update({'label':label})

In [17]:
# Convert char_based_id to token_based_id
# Find the corossponding token id after input being tokenized
add_token_positions(train_encodings, label_train)
add_token_positions(val_encodings, label_valid)
add_token_positions(test_encodings, label_test)

  0%|          | 0/25142 [00:00<?, ?it/s]

  0%|          | 0/3143 [00:00<?, ?it/s]

  0%|          | 0/3143 [00:00<?, ?it/s]

In [18]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'agreement', 'label'])

In [19]:
print(train['q\''].iloc[0])
print()
print(train['r\''].iloc[0])

When this occurs , you are much more likely to get misuse of firearms .

are examples of this actually taking place to warrant it being a problem that needs addressing


## Dataset

In [20]:
class qrDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

In [21]:
train_dataset = qrDataset(train_encodings)
val_dataset = qrDataset(val_encodings)
test_dataset = qrDataset(test_encodings)

In [22]:
next(iter(train_dataset))

/tmp/ipykernel_98330/3310948353.py:6: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484808560/work/torch/csrc/utils/tensor_new.cpp:201.)
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'input_ids': tensor([  101,  2043,  2023,  5158,  1010,  2017,  2024,  2172,  2062,  3497,
          2000,  2131, 28616,  8557,  1997, 13780,  1012,   102, 21090,  1024,
          1998,  2129,  2411,  2038,  2023,  2941,  4158,  1029,  2672,  2045,
          2024,  4973,  1997,  2023,  2941,  2635,  2173,  2000, 10943,  2009,
          2108,  1037,  3291,  2008,  3791, 12786,  1029,  2030,  2003,  2023,
          2074,  2115,  2219,  3167,  6772,  1029,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

## Model

In [23]:
from transformers import BertModel

class myModel(torch.nn.Module):

    def __init__(self):

        super(myModel, self).__init__()

        #self.Agree_model = BertModel.from_pretrained("nreimers/MiniLM-L6-H384-uncased")
        self.Disagree_model = BertModel.from_pretrained("nreimers/MiniLM-L6-H384-uncased")
        self.fc = nn.Linear(384, len(label_to_id))

    def forward(self, input_ids, attention_mask, token_type_ids, agreement):

        output = self.Disagree_model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, return_dict=True)
        logits = output[0]
        out = self.fc(logits)
        return out



## Training

In [24]:
from transformers import AdamW
from tqdm import tqdm

# Set GPU / CPU
device = torch.device('cuda:1') if torch.cuda.is_available() else torch.device('cpu')
# Put model on device
model = myModel().to(device)

optim = AdamW(model.parameters(), lr=1e-4)

/home/joeyliang/anaconda3/envs/nlp/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [25]:
# Pack data into dataloader by batch
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [26]:
training_epoch = 10

In [27]:
loss_fct = CrossEntropyLoss()

In [28]:
def evaluate(valid_loader):
    model.eval()
    running_loss = 0.0

    with torch.no_grad():
        loop = tqdm(valid_loader, leave=True)
        for batch_id, batch in enumerate(loop):
            input_ids      = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            label          = batch['label'].to(device)
            agreement      = batch['agreement'].to(device)
            # model output
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, agreement=agreement)
            
            loss = loss_fct(outputs.transpose(1,2), label.transpose(1,2)) 
            running_loss += loss.item()
            if batch_id % 30 == 0 and batch_id != 0:
                #print('Validation Epoch {} Batch {} Loss {:.4f}'.format(
                #    batch_id + 1, batch_id, running_loss / 30))
                running_loss = 0.0

        print("evaluate loss: ", loss)

In [ ]:
for epoch in range(training_epoch):
    model.train()
    running_loss = 0.0

    loop = tqdm(train_loader, leave=True)
    for batch_id, batch in enumerate(loop):
        # reset
        optim.zero_grad()
        
        input_ids      = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        label          = batch['label'].to(device)
        agreement          = batch['agreement'].to(device)
        # model output
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, agreement=agreement)
        loss = loss_fct(outputs.transpose(1,2), label.transpose(1,2))

        # calculate loss
        loss.backward()
        # update parameters
        optim.step()

        running_loss += loss.item()
        if batch_id % 50 == 0 and batch_id != 0:
            #print('Epoch {} Batch {} Loss {:.4f}'.format(
            #    batch_id + 1, batch_id, running_loss / 50))
            running_loss = 0.0

        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())
    evaluate(valid_loader)
torch.save(model.state_dict(), 'aicup_model_disagree_only_model')

Epoch 0:  94%|████████████████▉ | 1477/1572 [07:35<00:29,  3.19it/s, loss=0.127]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

100%|█████████████████████████████████████████| 197/197 [00:21<00:00,  9.38it/s]


evaluate loss:  tensor(0.1396, device='cuda:1')


Epoch 4:  47%|████████▉          | 744/1572 [03:53<04:19,  3.20it/s, loss=0.141]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Epoch 7:  31%|█████▌            | 482/1572 [02:31<05:46,  3.15it/s, loss=0.0993]

## Predict

In [36]:
def get_tag_indexes_from_scores(scores):
    import numpy as np
    predicted = []
    for sen in scores:
        pred_word = []
        for word in sen:
            pred_word.append(int(np.argmax(word)))
        predicted.append(pred_word)
    return predicted        

In [47]:
def predict(test_loader):

    model.eval()
    q_sub_output, r_sub_output = [],[]

    loop = tqdm(test_loader, leave=True)
    for batch_id, batch in enumerate(loop):
        input_ids      = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        label          = batch['label'].to(device)
        agreement      = batch['agreement'].to(device)
        # model output
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, agreement=agreement)
        outputs = get_tag_indexes_from_scores(torch.tensor(outputs, dtype=torch.float32).cpu().detach().numpy())
        outputs = torch.tensor(outputs, dtype=torch.long)
        for ids, labs in zip(input_ids, outputs):
            r = False
            q_sub=[]
            r_sub=[]
            for i in range(512):
                if labs[i]!=label_to_id['pad']:
                    if labs[i]==label_to_id['q_agreement']:
                        q_sub.append(ids[i])
                    elif labs[i]==label_to_id['r_agreement']:
                        r_sub.append(ids[i])
                
                
            q_sub_output.append(tokenizer.decode(q_sub))
            r_sub_output.append(tokenizer.decode(r_sub))
    return q_sub_output, r_sub_output



In [48]:
q_sub_output, r_sub_output = predict(test_loader)

  0%|                                                   | 0/197 [00:00<?, ?it/s]/tmp/ipykernel_98330/4058985228.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs = get_tag_indexes_from_scores(torch.tensor(outputs, dtype=torch.float32).cpu().detach().numpy())
100%|█████████████████████████████████████████| 197/197 [00:30<00:00,  6.43it/s]


In [49]:
test['q_sub'] = q_sub_output
test['r_sub'] = r_sub_output

In [50]:
test.head()

,id,q,r,s,q',r',q_sub,r_sub
24260,6423,What about the massacre of civilians in Bosnia...,DISAGREE:It was obvious that this stupid argum...,DISAGREE,Would n't it have been better if they had more...,It was obvious that this stupid argument comes...,what about the massacre of civilians in bosnia...,was obvious that this stupid argument comes up...
24355,6461,"No I listened , but I was wondering what other...",DISAGREE:You are perhaps thinking of the Mille...,DISAGREE,"No I listened , but I was wondering what other...",You are perhaps thinking of the Miller-Urrey e...,don't know how old you are nor if you can reme...,
6753,1802,With all the medical technology a child should...,DISAGREE:Then why does the following occur ?,DISAGREE,With all the medical technology a child should...,Then why does the following occur ?,with all the medical technology a child should...,then why does the following occur?
28600,7486,"There is n't an 'evolution mod ' , its just Ad...",DISAGREE:So since you admit there 's been a ``...,DISAGREE,"There has been a radical change in biology ,",do you still think neo-darwinism explains evol...,evolution mod,since you admit there's been a ` ` radical cha...
16077,4280,I take an American view because it was once il...,DISAGREE:Well it 's been illegal in other plac...,DISAGREE,I take an American view because it was once il...,I suggest in the context of this discussion yo...,i take an american view because it was once il...,well it's been illegal in other places and is ...


## Grading

In [51]:
def nltk_token_string(sentence):
    # print(sentence)
    tokens = nltk.word_tokenize(sentence)
    for i in range(len(tokens)):
        if len(tokens[i]) == 1:
            tokens[i] = re.sub(r"[!\"#$%&\'()*\+, -.\/:;<=>?@\[\\\]^_`{|}~]", '', tokens[i])
    while '' in tokens:
        tokens.remove('')
    tokens = ' '.join(tokens)
    return tokens

In [52]:
def lcs(X, Y):
    X_, Y_ = [], []
    
    X_ = nltk_token_string(X)
    Y_ = nltk_token_string(Y)

    m = len(X_)
    n = len(Y_)
 
    # declaring the array for storing the dp values
    L = [[None]*(n + 1) for i in range(m + 1)]
 
    """Following steps build L[m + 1][n + 1] in bottom up fashion
    Note: L[i][j] contains length of LCS of X[0..i-1]
    and Y[0..j-1]"""
    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0 or j == 0 :
                L[i][j] = 0
            elif X_[i-1] == Y_[j-1]:
                L[i][j] = L[i-1][j-1]+1
            else:
                L[i][j] = max(L[i-1][j], L[i][j-1])
 
    # L[m][n] contains the length of LCS of X[0..n-1] & Y[0..m-1]
    return L[m][n]


def acc(full, sub):
    common = lcs(full, sub)
    union = len(full) + len(sub) - common
    accuracy = float(common/union)

    return accuracy

In [53]:
q_acc_sum = 0
r_acc_sum = 0
for i in range(test.shape[0]):
    q_accuracy = acc(test.iloc[i]["q'"], test.iloc[i]['q_sub'])
    r_accuracy = acc(test.iloc[i]["r'"], test.iloc[i]['r_sub'])

    q_acc_sum += q_accuracy
    r_acc_sum += r_accuracy

print("q accuracy: ", q_acc_sum/test.shape[0])
print("r accuracy: ", r_acc_sum/test.shape[0])

q accuracy:  0.5080239760286445
r accuracy:  0.43067534601260304
